In [3]:
model_xgb_final = xgb.Booster()
model_xgb_final.load_model('li_intent_model_2025-10-07.json')

# Get feature importance
feature_importance = model_xgb_final.get_score(importance_type='gain')

# Convert to DataFrame
importance_df = pd.DataFrame(list(feature_importance.items()), columns=['Feature', 'Gain Value'])
importance_df = importance_df.sort_values('Gain Value', ascending=False)
importance_df['Feature Importance'] = importance_df['Gain Value'] / importance_df['Gain Value'].sum()
# Save to CSV
filename = 'abli_feature_importance_gradient.csv'
importance_df.to_csv(filename, index=False)

print(f"Feature importance saved to {filename}")

Feature importance saved to abli_feature_importance_gradient.csv


In [2]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import xgboost as xgb
import pandas_gbq
import shap
import seaborn as sns
import cloudpickle as pkl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import itertools

from google.cloud import bigquery, storage
from matplotlib import pyplot as plt
from joblib import Parallel, delayed
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.preprocessing import LabelEncoder
from box import Box
from scipy.stats import chi2_contingency, f_oneway
from statsmodels.formula.api import ols
from joblib import Parallel, delayed
from tqdm import tqdm
from itertools import product

client = bigquery.Client()
store = storage.Client()

## Create Model Training-OOT base

In [2]:
QUERY = """
CREATE OR REPLACE TABLE `abcd-dataplatform.abcd_data_science_app.li_intent_model_combined_features`
PARTITION BY snapshot_partition
AS (
    WITH cs_base AS (
        SELECT
            *
        FROM
            `abcd-dataplatform.abcd_data_science_app.li_intent_model_targets`
    ),

    cashflow_features AS (
        SELECT
            *
        FROM
            `abcd-dataplatform.abcd_data_science_app.abhi_intent_model_cashflow_features_raw`
    ),
    
    bank_features AS (
        SELECT
            *
        FROM
            `abcd-dataplatform.abcd_data_science_app.abhi_intent_model_bank_features_raw`
    ),
    
    loan_features AS (
        SELECT
            *
        FROM
            `abcd-dataplatform.abcd_data_science_app.abhi_intent_model_loan_features_raw`
    ),
    
    credit_card_features AS (
        SELECT
            *
        FROM
            `abcd-dataplatform.abcd_data_science_app.abhi_intent_model_credit_card_features_raw`
    ),
    
    investment_features AS (
        SELECT
            *
        FROM
            `abcd-dataplatform.abcd_data_science_app.abhi_intent_model_investment_features_raw`
    ),
    
    utility_features AS (
        SELECT
            *
        FROM
            `abcd-dataplatform.abcd_data_science_app.abhi_intent_model_utility_features_raw`
    ),
    
    spend_features AS (
        SELECT
            *
        FROM
            `abcd-dataplatform.abcd_data_science_app.abhi_intent_model_topspend_features_raw`
    ),
    
    insurance_features AS (
        SELECT
            *
        FROM
            `abcd-dataplatform.abcd_data_science_app.abhi_intent_model_insurance_features_raw`
    ),
    
    Anagog_features AS (
        SELECT 
        * 
        FROM `abcd-dataplatform.abcd_data_science_app.hi_intent_models_anagog_features_raw`
    ),
        
    inapp_features AS(
        SELECT 
        * 
        FROM `abcd-dataplatform.abcd_data_science_app.hi_intent_models_inapp_dha_interacted_features_raw`
    ),
        
    cust_age_features AS(
    SELECT 
    *
    FROM `abcd-dataplatform.abcd_data_science_app.hi_intent_models_cust_age_pin_features_raw`
    ),
    
--    exp_loan_features AS (
--  SELECT 
--    CUSTOMER_ID AS mobilenumber,
--    * EXCEPT(CUSTOMER_ID)
--    FROM `abcd-dataplatform.abcd_data_science_app.abhi_intent_model_bureau_loan_amount_features_raw`
--    ),
    
        
        
    joined_base AS (
        SELECT
            base.*,
            cf.* EXCEPT(mobilenumber, snapshot_partition), 
            bf.* EXCEPT(mobilenumber, snapshot_partition),
            lf.* EXCEPT(mobilenumber, snapshot_partition),
            ccf.* EXCEPT(mobilenumber, snapshot_partition),
            invf.* EXCEPT(mobilenumber, snapshot_partition),
            uf.* EXCEPT(mobilenumber, snapshot_partition),
            sf.* EXCEPT(mobilenumber, snapshot_partition),
            insurf.* EXCEPT(mobilenumber, snapshot_partition),
            anagog_f.* EXCEPT(customer_id, mobilenumber, snapshot_partition),
            inapp_f.* EXCEPT(customer_id, mobilenumber, snapshot_partition),
            cust_age_f.* EXCEPT(customer_id, mobilenumber, snapshot_partition),
--            exp_loan_f.* EXCEPT(mobilenumber, snapshot_partition)
            
            
            
        FROM
            cs_base AS base
        LEFT JOIN
            cashflow_features AS cf
        USING(mobilenumber, snapshot_partition)
        
        LEFT JOIN
            bank_features AS bf
        USING(mobilenumber, snapshot_partition) 
        
        LEFT JOIN
            loan_features AS lf
        USING(mobilenumber, snapshot_partition) 
        
        
        LEFT JOIN
            credit_card_features AS ccf
        USING(mobilenumber, snapshot_partition)
        
        LEFT JOIN
            investment_features AS invf
        USING(mobilenumber, snapshot_partition)
        
        LEFT JOIN
            utility_features AS uf
        USING(mobilenumber, snapshot_partition)
        
         LEFT JOIN
            spend_features AS sf
        USING(mobilenumber, snapshot_partition)
        
        LEFT JOIN
            insurance_features AS insurf
        USING(mobilenumber, snapshot_partition)
        
        LEFT JOIN
            Anagog_features AS anagog_f
        USING(mobilenumber, snapshot_partition)
        
        LEFT JOIN
            inapp_features AS inapp_f
        USING(mobilenumber, snapshot_partition)
        
        LEFT JOIN
            cust_age_features AS cust_age_f
        USING(mobilenumber, snapshot_partition)
        
--        LEFT JOIN
--            exp_loan_features AS exp_loan_f
--        USING(mobilenumber, snapshot_partition)
)
    
    SELECT *
    FROM joined_base

)
"""

# sms_grad_features = client.query(QUERY).to_dataframe()
# sms_grad_features.info()

client.query(QUERY).result()

## Feature Selection

### Stratified Sample

In [3]:
%%bigquery

CREATE OR REPLACE TABLE
  `abcd-dataplatform.abcd_data_science_app.li_base_data` AS (
  WITH
    counts AS (
    SELECT
      snapshot_partition,
      abli_target,
      COUNT(*) AS cnt
    FROM
      `abcd-dataplatform.abcd_data_science_app.li_intent_model_combined_features`
    WHERE
      snapshot_partition IN ('2024-12-31',
        '2025-01-31',
        '2025-02-28',
        '2025-03-31')
    GROUP BY
      snapshot_partition,
      abli_target ),
    with_sample AS (
    SELECT
      f.*,
      ROW_NUMBER() OVER (PARTITION BY snapshot_partition, final_target ORDER BY RAND()) AS rn
    FROM
      `abcd-dataplatform.abcd_data_science_app.li_intent_model_combined_features` f
    WHERE
      snapshot_partition IN ('2024-12-31',
        '2025-01-31',
        '2025-02-28',
        '2025-03-31') )
  SELECT
    w.* EXCEPT(rn)
  FROM
    with_sample w
  JOIN
    counts c
  ON
    w.snapshot_partition = c.snapshot_partition
    AND w.final_target = c.abli_target
  WHERE
    rn <= SAFE_CAST(c.cnt * 0.1 AS INT64) 
);

Query is running:   0%|          |

""


In [2]:
%%bigquery hi_base_data

SELECT * FROM `abcd-dataplatform.abcd_data_science_app.li_base_data`

Query is running:   0%|          |

Downloading:   0%|          |

In [3]:
hi_base_data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1776619 entries, 0 to 1776618
Columns: 866 entries, customer_id to mobile_device_price
dtypes: Int64(89), datetime64[us](1), dbdate(1), float64(755), object(20)
memory usage: 12.8 GB


In [4]:
hi_base_data.final_target.sum() / hi_base_data.shape[0]

0.0005972017635745199

In [5]:
hi_base_data[['tier', 'final_target']].value_counts(dropna=False).sort_index()

tier   final_target
Tier1  0               247330
       1                  209
Tier2  0               449208
       1                  261
Tier3  0               761394
       1                  464
NaN    0               317626
       1                  127
Name: count, dtype: int64

In [2]:
date_cols = [
    'latest_txn_date_ALL', 
    'latest_txn_date_L1M', 
    'latest_txn_date_L3M', 
    'latest_txn_date_L6M', 
    'latest_txn_date_L9M',
    'latest_txn_date_L12M'
]

categorical_cols = [
    'most_frequent_bank', 
    'bank_with_highest_avg_balance', 
    'max_spend_category', 
    'bank_w_highest_cc_limit',
    'bank_w_highest_loan_emi_due',
    'IsGymLover',
    'IsSingle',
    'IsParent',
    'employment_type',
    'tier',
    'age_bracket'
]

In [3]:
xgb_feature_params = {
    'objective': 'binary:logistic',
    'max_depth': 3,
    'n_jobs': 24,
    'learning_rate': 0.01,
    # 'n_estimators': 500,
    # 'gamma': 2,
    'subsample': 0.75,
    'colsample_bytree': 0.8,
    # 'lambda': 0.1,
    # 'alpha': 0.1
    'eval_metric': 'logloss'
    # 'scale_pos_weight': 666
}

### Feature Selection Process

In [8]:
print('Modifying Date columns:')
for date_col in date_cols:
    if date_col in hi_base_data.columns:
        print(f'\t{date_col}')
        hi_base_data[date_col] = (pd.to_datetime(hi_base_data.loc[:, 'snapshot_partition']) + pd.DateOffset(months=1) - pd.DateOffset(days=1) - pd.to_datetime(hi_base_data.loc[:, date_col])).dt.days

print('\nModifying Categorical columns:')
# Ensure categorical columns have the dtype 'category'
for col in categorical_cols:
    if col in hi_base_data.columns:
        print(f'\t{col}')
        hi_base_data[col] = hi_base_data[col].astype('category')

Modifying Date columns:
	latest_txn_date_ALL
	latest_txn_date_L1M
	latest_txn_date_L3M
	latest_txn_date_L6M
	latest_txn_date_L9M
	latest_txn_date_L12M

Modifying Categorical columns:
	most_frequent_bank
	bank_with_highest_avg_balance
	max_spend_category
	bank_w_highest_cc_limit
	bank_w_highest_loan_emi_due
	IsGymLover
	IsSingle
	IsParent
	employment_type
	tier
	age_bracket


In [9]:
# Prepare the training data by filtering out unwanted columns
train_features = hi_base_data.loc[:, ~(hi_base_data.columns.str.contains('customer_id') |
                                  hi_base_data.columns.str.contains('mobilenumber') |
                                  hi_base_data.columns.str.contains('registration_date') |
                                  hi_base_data.columns.str.contains('app_target') |
                                  hi_base_data.columns.str.contains('final_target') |
                                  hi_base_data.columns.str.contains('sms_target') |
                                  hi_base_data.columns.str.contains('mobile_device_price') |
                                  hi_base_data.columns.str.contains('snapshot_partition'))]

In [10]:
train_features = train_features.drop(columns=['age', 'cashflow_num_months'])

In [11]:
target_column = 'abli_target'
y_train = hi_base_data[target_column]
train_features = train_features.drop(columns=[target_column])

# Create a DMatrix for training
dtrain = xgb.DMatrix(train_features, label=y_train, enable_categorical=True,)

In [12]:
%%time
# Train the model using XGBoost's train function
model_xgb = xgb.train(params=xgb_feature_params, dtrain=dtrain, num_boost_round=500)

CPU times: user 21min 40s, sys: 0 ns, total: 21min 40s
Wall time: 1min 7s


In [13]:
%%time
explainer = shap.TreeExplainer(model_xgb)
shap_values = explainer.shap_values(dtrain)

CPU times: user 22min 45s, sys: 0 ns, total: 22min 45s
Wall time: 1min 10s


In [14]:
# Convert SHAP values to a DataFrame with column names from the DMatrix
shap_abs = pd.DataFrame(shap_values, columns=train_features.columns).abs().mean()

# Filter columns where the mean absolute SHAP value is greater than 0.001
filtered_columns = shap_abs[shap_abs > 0.001].index.tolist()

# Output the number of filtered columns
print(f"Number of filtered columns: {len(filtered_columns)}")
print(f"Number of filtered columns: {len(set(filtered_columns))}")

Number of filtered columns: 275
Number of filtered columns: 275


In [15]:
# Define mandatory columns you want to keep
forced_columns = ['tier', 'age_bracket', 'dha_interacted', 'IsSingle', 'IsParent']
forced_columns = [col for col in forced_columns if shap_abs.loc[col] > 0.0001]
forced_columns = ['IsParent']


In [16]:
filtered_columns.extend([col for col in forced_columns if col not in filtered_columns])

In [17]:
final_columns = [col for col in filtered_columns if col not in categorical_cols]

In [18]:
%%time
columns = final_columns
ddf = dd.from_pandas(train_features[final_columns], npartitions=32)
correlation_matrix = ddf.corr().compute()

CPU times: user 1h 19min 27s, sys: 27min 17s, total: 1h 46min 45s
Wall time: 8min 29s


In [19]:
cutoff = 0.6
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
correlated_pairs = [(column, row) for column in upper_triangle.columns for row in upper_triangle.index if upper_triangle.at[row, column] > cutoff]

len(correlated_pairs)

2149

In [20]:
drop_set = set()
for column, row in correlated_pairs:
    if shap_abs.loc[column] > shap_abs.loc[row]:
        drop_set.add(row)
    else:
        drop_set.add(column)

drop_set = list(drop_set)
filtered_features = [column for column in filtered_columns if column not in drop_set]
len(filtered_features)

62

In [21]:
def compute_pearson_corr(col1, col2, data):
    return data[col1].corr(data[col2], min_periods=1)

# Function to compute Cramér's V for categorical features
def cramers_v(col1, col2, data):
    contingency_table = pd.crosstab(data[col1], data[col2])
    chi2, _, _, _ = chi2_contingency(contingency_table)
    n = contingency_table.sum().sum()
    r, k = contingency_table.shape
    return np.sqrt(chi2 / (n * (min(k - 1, r - 1))))

# Function to compute Eta Squared for numerical vs categorical
def compute_eta_squared(col1, col2, data):
    # Drop rows with missing values in either column
    sub = data[[col1, col2]].dropna()
    # If no data or only one category or zero variance, return 0
    if sub.empty or sub[col2].nunique() < 2 or sub[col1].var() == 0:
        return 0.0

    # Calculate total sum of squares (SST)
    overall_mean = sub[col1].mean()
    sst = ((sub[col1] - overall_mean) ** 2).sum()

    # Calculate sum of squares between groups (SSB)
    ssb = 0.0
    for cat in sub[col2].unique():
        group = sub[col1][sub[col2] == cat]
        # group always has length > 0 since sub filtered
        group_mean = group.mean()
        ssb += len(group) * (group_mean - overall_mean) ** 2

    # Eta squared is the ratio of explained variance to total variance
    return ssb / sst if sst > 0 else 0.0

# General function to compute association based on data types
def compute_association(col1, col2, data, categorical_cols):
    is_cat1 = col1 in categorical_cols
    is_cat2 = col2 in categorical_cols

    # Numerical vs Numerical
    if not is_cat1 and not is_cat2:
        return compute_pearson_corr(col1, col2, data)
    # Categorical vs Categorical
    elif is_cat1 and is_cat2:
        return cramers_v(col1, col2, data)
    # Numerical vs Categorical
    else:
        # Identify which is numerical and which is categorical
        num_col, cat_col = (col1, col2) if not is_cat1 else (col2, col1)
        return compute_eta_squared(num_col, cat_col, data)

# Assuming 'train_features' is the input dataframe, 'filtered_features' is numerical, and 'categorical_cols' lists categorical columns
columns = filtered_features
n_jobs = -1  # Use all available cores

# Generate the matrix in parallel
association_values = Parallel(n_jobs=n_jobs)(
    delayed(compute_association)(col1, col2, train_features, categorical_cols)
    for col1 in columns for col2 in columns
)

# Reshape the list into a square matrix
association_matrix = pd.DataFrame(
    np.array(association_values).reshape(len(columns), len(columns)),
    index=columns, columns=columns
)

In [22]:
# Set cutoff for high association
cutoff = 0.6
# Use upper triangle of the association matrix to avoid duplicate pairs
upper_triangle = association_matrix.where(np.triu(np.ones(association_matrix.shape), k=1).astype(bool))
# Find all feature pairs with association above the cutoff
correlated_pairs = [
    (col, row)
    for col in upper_triangle.columns
    for row in upper_triangle.index
    if upper_triangle.abs().at[row, col] > cutoff
]

print(f"Found {len(correlated_pairs)} highly associated pairs (cutoff={cutoff}).")

# Determine which features to drop based on SHAP importance
# Assuming 'shap_abs' is a Series of absolute SHAP values indexed by feature
drop_set = set()
for col, row in correlated_pairs:
    
    if col in forced_columns and row in forced_columns:
        if shap_abs.loc[col] > shap_abs.loc[row]:
            drop_set.add(row)
        else:
            drop_set.add(col)
    
    elif col in forced_columns:
        drop_set.add(row)
    elif row in forced_columns:
        drop_set.add(col)

    elif shap_abs.loc[col] > shap_abs.loc[row]:
        drop_set.add(row)
    else:
        drop_set.add(col)

# Final filtered feature list
drop_set = list(drop_set)
filtered_features = [col for col in filtered_features if col not in drop_set]

print(f"Dropped {len(drop_set)} features: {drop_set}")
print(f"Remaining features ({len(filtered_features)})")

Found 2 highly associated pairs (cutoff=0.6).
Dropped 2 features: ['min_monthly_surplus_L12M', 'avg_monthly_cc_due_balance_L12M']
Remaining features (60)


In [23]:
filtered_features

['min_total_monthly_expense_L12M',
 'max_total_monthly_expense_L12M',
 'max_monthly_surplus_L12M',
 'most_frequent_bank',
 'bank_with_highest_avg_balance',
 'sum_monthly_debit_L3M',
 'max_monthly_bankbalance_L6M',
 'latest_txn_date_L9M',
 'avg_monthly_cc_repayment',
 'avg_monthly_cc_due_balance_L1M',
 'avg_monthly_cc_due_amount_L3M',
 'bank_w_highest_cc_limit',
 'sum_total_credit_limit',
 'max_cc_due_amount',
 'avg_cc_repayment_L1M',
 'unpaid_cc_status_avg_due_amount_L1M',
 'fully_paid_cc_status_avg_due_amount_L3M',
 'fully_paid_cc_status_avg_repayment_L3M',
 'unknown_cc_status_avg_due_amount_L3M',
 'avg_monthly_investment_count_L6M',
 'provident_fund_investment_count_ALL',
 'avg_investment_amt_L3M',
 'min_investment_amt_L3M',
 'fixed_deposit_total_investment_amt_L6M',
 'provident_fund_avg_investment_amt_L6M',
 'min_utility_paid_amount',
 'avg_utility_paid_amount',
 'avg_utility_due_amount',
 'unique_payment_modes_count',
 'bills_utilities_avg_topspend_amount',
 'cash_withdrawal_avg_to

In [24]:
k = 20
shap_ranked = (
    shap_abs[shap_abs.index.isin(filtered_features)]
    .drop(forced_columns, errors="ignore")  # drop forced features to avoid duplication
    .sort_values(ascending=False)
)

remaining_slots = k - len(forced_columns)
top_remaining = shap_ranked.head(remaining_slots).index.tolist()
top_features = list(dict.fromkeys(forced_columns + top_remaining))  # preserve order, avoid dupes

top_features

['IsParent',
 'age_bracket',
 'bank_with_highest_avg_balance',
 'insurance_avg_topspend_amount',
 'max_total_monthly_expense_L12M',
 'latest_txn_date_L9M',
 'emi_repayment_avg_topspend_amount',
 'most_frequent_bank',
 'max_monthly_surplus_L12M',
 'max_monthly_bankbalance_L6M',
 'bank_w_highest_cc_limit',
 'min_investment_amt_L3M',
 'others_insurance_count_of_policies',
 'upi_avg_topspend_amount',
 'provident_fund_avg_investment_amt_L6M',
 'avg_utility_paid_amount',
 'sum_monthly_debit_L3M',
 'emi_repayment_count_topspend',
 'insurance_count_topspend',
 'avg_monthly_investment_count_L6M']

## Model Training

### Load Traning Data

In [5]:
#Including Bureau Features
top_features = [
    'bank_with_highest_avg_balance',
    'OVERDUE_BY_BALANCE_RATIO_SECURED_CURRENTLY_ACTIVE_SUM',
    'util_increase_count_L6M_AVG',
    'LOAN_AMT_NONMORTGAGE_CURRENTLY_INACTIVE_MAX',
    'most_frequent_bank',
    'AMT_OVERDUE_NONCC_REPORTED_L6M_MAX',
    'VINTAGE_MAX',
    'AMT_OVERDUE_UNSECURED_MAX',
    'LOAN_AMT_CC_MAX',
    'sum_monthly_credit_L9M',
    'LOAN_AMT_ALL_OLDER_L6M_SUM',
    'bank_w_highest_cc_limit',
    'latest_txn_date_L12M',
    'OVERDUE_BY_BALANCE_RATIO_ALL_OPENED_L12M_SUM',
    'AMT_OVERDUE_PL_CURRENTLY_ACTIVE_SUM',
    'COUNT_LOAN_PL_ALL_RATIO',
    'OVERDUE_BY_BALANCE_RATIO_SECURED_SUM',
    'UTIL_AMT_CC_SUM',
    'AMT_OVERDUE_UNSECURED_OPENED_L12M_CURRENTLY_ACTIVE_MAX',
    'LOAN_AMT_UNSECURED_OPENED_L12M_MAX'
]

In [ ]:
#Not Including Bureau Features
top_features = [
    'bank_w_highest_loan_emi_due',
    'unique_loan_accounts',
    'age_bracket',
    'bank_with_highest_avg_balance',
    'cashflow_num_months',
    'bank_w_highest_cc_limit',
    'insurance_avg_topspend_amount',
    'total_no_of_insurance_policies',
    'medical_avg_topspend_amount',
    'vehicle_expenses_avg_topspend_amount',
    'max_total_monthly_inflow_L12M',
    'most_frequent_bank',
    'avg_monthly_credit_L9M',
    'shopping_purchase_avg_topspend_amount',
    'avg_monthly_bankbalance_L12M',
    'gaming_avg_topspend_amount',
    'min_monthly_bankbalance_ALL',
    'tier',
    'min_total_credit_limit',
    'gaming_count_topspend'
]

In [74]:
top_features = [
    'tier',
    'age_bracket',
    'dha_interacted',
    'IsGymLover',
    'bank_with_highest_avg_balance',
    'cashflow_num_months',
    'most_frequent_bank',
    'bank_w_highest_cc_limit',
    'max_total_monthly_inflow_L12M',
    'avg_monthly_bankbalance_ALL',
    'medical_avg_topspend_amount',
    'emi_repayment_avg_topspend_amount',
    'insurance_avg_topspend_amount',
    'vehicle_expenses_avg_topspend_amount',
    'min_monthly_bankbalance_ALL',
    'avg_monthly_utility_due_amount_L12M',
    'total_no_of_insurance_policies',
    'gaming_count_topspend',
    'sum_total_credit_limit',
    'groceries_avg_topspend_amount'
]

In [2]:
top_features = [
    'tier',
    'age_bracket',
    'dha_interacted',
    'bank_with_highest_avg_balance',
    'cashflow_num_months',
    'max_total_monthly_inflow_L12M',
    'most_frequent_bank',
    'bank_w_highest_cc_limit',
    'avg_monthly_bankbalance_ALL',
    'insurance_avg_topspend_amount',
    'medical_avg_topspend_amount',
    'emi_repayment_avg_topspend_amount',
    'vehicle_expenses_avg_topspend_amount',
    'min_monthly_bankbalance_ALL',
    'total_no_of_insurance_policies',
    'avg_monthly_utility_due_amount_L12M',
    'gaming_avg_topspend_amount',
    'avg_monthly_utility_paid_amount',
    'sum_total_credit_limit',
    'max_cc_repayment_L12M'
]

In [4]:
top_features = [
    'IsParent',
    'age_bracket',
    'bank_with_highest_avg_balance',
    'insurance_avg_topspend_amount',
    'max_total_monthly_expense_L12M',
    'latest_txn_date_L9M',
    'emi_repayment_avg_topspend_amount',
    'most_frequent_bank',
    'max_monthly_surplus_L12M',
    'max_monthly_bankbalance_L6M',
    'bank_w_highest_cc_limit',
    'min_investment_amt_L3M',
    'others_insurance_count_of_policies',
    'upi_avg_topspend_amount',
    'provident_fund_avg_investment_amt_L6M',
    'avg_utility_paid_amount',
    'sum_monthly_debit_L3M',
    'emi_repayment_count_topspend',
    'insurance_count_topspend',
    'avg_monthly_investment_count_L6M'
]

In [5]:
feature_query = ', '.join(top_features)
feature_query

'IsParent, age_bracket, bank_with_highest_avg_balance, insurance_avg_topspend_amount, max_total_monthly_expense_L12M, latest_txn_date_L9M, emi_repayment_avg_topspend_amount, most_frequent_bank, max_monthly_surplus_L12M, max_monthly_bankbalance_L6M, bank_w_highest_cc_limit, min_investment_amt_L3M, others_insurance_count_of_policies, upi_avg_topspend_amount, provident_fund_avg_investment_amt_L6M, avg_utility_paid_amount, sum_monthly_debit_L3M, emi_repayment_count_topspend, insurance_count_topspend, avg_monthly_investment_count_L6M'

In [6]:
QUERY = f"""
    SELECT DISTINCT
        customer_id, snapshot_partition, abli_target AS target, {feature_query}
    FROM
        `abcd-dataplatform.abcd_data_science_app.li_intent_model_combined_features`
    WHERE
        snapshot_partition < '2025-04-30'
"""

train_set = client.query(QUERY).to_dataframe()
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17766174 entries, 0 to 17766173
Data columns (total 23 columns):
 #   Column                                 Dtype  
---  ------                                 -----  
 0   customer_id                            object 
 1   snapshot_partition                     dbdate 
 2   target                                 Int64  
 3   IsParent                               object 
 4   age_bracket                            object 
 5   bank_with_highest_avg_balance          object 
 6   insurance_avg_topspend_amount          float64
 7   max_total_monthly_expense_L12M         float64
 8   latest_txn_date_L9M                    object 
 9   emi_repayment_avg_topspend_amount      float64
 10  most_frequent_bank                     object 
 11  max_monthly_surplus_L12M               float64
 12  max_monthly_bankbalance_L6M            float64
 13  bank_w_highest_cc_limit                object 
 14  min_investment_amt_L3M                 float64
 

In [7]:
train_set.groupby(['customer_id', 'snapshot_partition']).ngroups

17766174

In [8]:
train_set = train_set[train_set[top_features].notna().any(axis=1)].reset_index(drop=True)
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14790426 entries, 0 to 14790425
Data columns (total 23 columns):
 #   Column                                 Dtype  
---  ------                                 -----  
 0   customer_id                            object 
 1   snapshot_partition                     dbdate 
 2   target                                 Int64  
 3   IsParent                               object 
 4   age_bracket                            object 
 5   bank_with_highest_avg_balance          object 
 6   insurance_avg_topspend_amount          float64
 7   max_total_monthly_expense_L12M         float64
 8   latest_txn_date_L9M                    object 
 9   emi_repayment_avg_topspend_amount      float64
 10  most_frequent_bank                     object 
 11  max_monthly_surplus_L12M               float64
 12  max_monthly_bankbalance_L6M            float64
 13  bank_w_highest_cc_limit                object 
 14  min_investment_amt_L3M                 float64
 

### Model Training Process

In [9]:
date_cols_filtered = [col for col in top_features if col in date_cols]
categorical_cols_filtered = [col for col in top_features if col in categorical_cols]

In [10]:
def create_dmatrix(df, date_cols, categorical_cols, label=None):
    global train_features
    
    # Convert date columns to days between dates
    if date_cols:
        for date_col in date_cols:
            df[date_col] = (pd.to_datetime(df.loc[:, 'snapshot_partition']) + pd.DateOffset(months=1) - pd.DateOffset(days=1) - pd.to_datetime(df.loc[:, date_col])).dt.days

    # Label encode categorical columns
    if categorical_cols:
        for col in categorical_cols:
            df[col]=df[col].astype('category')
            

    # Prepare the training data by filtering out unwanted columns
    train_features = df.loc[:, ~(df.columns.str.contains('customer_id') |
                                 df.columns.str.contains('mobilenumber') |
                                 df.columns.str.contains('registration_date') |
                                 df.columns.str.contains('digital_flag') |
                                 df.columns.str.contains('app_flag') |
                                 df.columns.str.contains('snapshot_partition'))]
    
    # Create a DMatrix for training
    dmat = xgb.DMatrix(train_features, label=label, enable_categorical=True)
    return dmat

In [11]:
ginis = []
train_ginis = []
train_events = []
events = []
num_events = []
train_num_events = []
models = []
split_indices = []

# StratifiedShuffleSplit configuration
n_splits = 5  # Number of splits
test_size = 0.2  # 20% for validation

# Assuming `X` is your features and `y` is your target
sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=42)

# Track accuracy scores across splits (use a different metric if needed)
scores = []
previous_model = None
eval_pass = 0

for i, (train_index, val_index) in enumerate(tqdm(sss.split(train_set[top_features + ['snapshot_partition']], train_set['target']), desc='Evaluation pass', total=n_splits)):
    
    split_indices.append((train_index, val_index))
    X_train, X_val = train_set.loc[train_index, top_features + ['snapshot_partition']], train_set.loc[val_index, top_features + ['snapshot_partition']]
    y_train, y_val = train_set.loc[train_index, 'target'], train_set.loc[val_index, 'target']

    # Convert to DMatrix for XGBoost
    dtrain = create_dmatrix(X_train, date_cols=date_cols_filtered, categorical_cols=categorical_cols_filtered, label=y_train)
    dval = create_dmatrix(X_val, label=y_val, date_cols=date_cols_filtered, categorical_cols=categorical_cols_filtered)

    # Train the model
    evals = [(dtrain, 'train'), (dval, 'val')]
    model = xgb.train(
        params=xgb_feature_params,
        dtrain=dtrain,
        num_boost_round=500,
        evals=evals,
        early_stopping_rounds=50,
        verbose_eval=False
        # xgb_model=previous_model
    )
    
    models.append(model)
#     previous_model = model_dart

    # Predict and calculate accuracy (replace with a different evaluation metric if needed)
    y_pred = model.predict(dval)
    y_pred_binary = (y_pred > 0.5).astype(int)  # Assuming a binary classification problem
    y_true = dval.get_label()

    accuracy = accuracy_score(y_val, y_pred_binary)
    auc = roc_auc_score(y_true, y_pred)
    ginis.append(2*auc - 1)

    y_pred = model.predict(dtrain)
    y_pred_binary = (y_pred > 0.5).astype(int)  # Assuming a binary classification problem
    y_true = dtrain.get_label()
    auc = roc_auc_score(y_true, y_pred)
    train_ginis.append(2*auc - 1)
    scores.append(accuracy)
    
    train_events.append(y_train.sum() / X_train.shape[0])
    events.append(y_val.sum() / X_val.shape[0])

    train_num_events.append(y_train.sum())
    num_events.append(y_val.sum())
    
    eval_pass += 1
    # print(f"Evaluation Pass {eval_pass} Completed!")

# Print average accuracy
print(f"Average Accuracy across {n_splits} folds: {np.mean(scores):.4f}")

Evaluation pass: 100%|██████████| 5/5 [22:50<00:00, 274.10s/it]

Average Accuracy across 5 folds: 0.9994


### Model Metrics and Selection

In [12]:
cross_val_train_ginis = []
cross_val_ginis = []

pbar = tqdm(product(range(len(models)), range(len(split_indices))),
            total=len(models)*len(split_indices),
            desc="Model x Fold")

for i, j in pbar:
    
    train_index = split_indices[j][0]
    val_index = split_indices[j][1]

    X_train, X_val = train_set.loc[train_index, top_features + ['snapshot_partition']], train_set.loc[val_index, top_features + ['snapshot_partition']]
    y_train, y_val = train_set.loc[train_index, 'target'], train_set.loc[val_index, 'target']

    # Convert to DMatrix for XGBoost
    dtrain = create_dmatrix(X_train, date_cols=date_cols_filtered, categorical_cols=categorical_cols_filtered, label=y_train)
    dval = create_dmatrix(X_val, label=y_val, date_cols=date_cols_filtered, categorical_cols=categorical_cols_filtered)

    y_pred = models[i].predict(dval)
    y_true = dval.get_label()
    auc = roc_auc_score(y_true, y_pred)
    val_gini = 2*auc - 1
    cross_val_ginis.append(val_gini)

    y_pred = models[i].predict(dtrain)
    y_true = dtrain.get_label()
    auc = roc_auc_score(y_true, y_pred)
    train_gini = 2*auc - 1
    cross_val_train_ginis.append(train_gini)

    pbar.set_postfix({
        "Model": i+1,
        "Fold": j+1,
        "Val Gini": f"{val_gini*100:.2f}%",
        "Train Gini": f"{train_gini*100:.2f}%"
    })
    

Model x Fold: 100%|██████████| 25/25 [56:35<00:00, 135.81s/it, Model=5, Fold=5, Val Gini=78.91%, Train Gini=80.44%]


In [13]:
print(f""" -- FOR VALIDATION --
Average Gini = {np.mean(cross_val_ginis) * 100:.2f}%, 
Maximum Gini = {np.max(cross_val_ginis) * 100:.2f}%
Minimum Gini = {np.min(cross_val_ginis) * 100:.2f}%
""")

 -- FOR VALIDATION --
Average Gini = 79.34%, 
Maximum Gini = 80.86%
Minimum Gini = 77.98%



In [14]:
print(f""" -- FOR TRAINING --
Average Gini = {np.mean(cross_val_train_ginis) * 100:.2f}%, 
Maximum Gini = {np.max(cross_val_train_ginis) * 100:.2f}%
Minimum Gini = {np.min(cross_val_train_ginis) * 100:.2f}%
""")

 -- FOR TRAINING --
Average Gini = 80.18%, 
Maximum Gini = 80.59%
Minimum Gini = 79.63%



In [15]:
len(cross_val_train_ginis)

25

In [16]:
# Define number of folds (assumed constant for all models)
folds = 5
models_count = len(cross_val_train_ginis) // folds  # Total number of models

# Compute averages
average_ginis = [
    Box({
        "model": f"Model {i+1}",
        "avg_train_gini": sum(cross_val_train_ginis[i * folds : (i + 1) * folds]) / folds,
        "avg_val_gini": sum(cross_val_ginis[i * folds : (i + 1) * folds]) / folds
    })
    for i in range(models_count)
]
# Print results
for avg in average_ginis:
    print(f"{avg['model']}: Train Gini = {avg['avg_train_gini'] * 100:.2f}%, Val Gini = {avg['avg_val_gini'] * 100:.2f}%")

Model 1: Train Gini = 80.22%, Val Gini = 79.36%
Model 2: Train Gini = 80.27%, Val Gini = 79.30%
Model 3: Train Gini = 79.98%, Val Gini = 79.36%
Model 4: Train Gini = 80.18%, Val Gini = 79.30%
Model 5: Train Gini = 80.28%, Val Gini = 79.37%


In [17]:
np.argmax(ginis)

0

In [18]:
stability = np.absolute([average_ginis[i].avg_train_gini - average_ginis[i].avg_val_gini for i in range(len(average_ginis))])

In [19]:
stability

array([0.00856169, 0.00967157, 0.00616504, 0.00876319, 0.00911845])

In [20]:
print(f""" -- MODEL STABILITY --
Average Gini Difference = {np.mean(stability) * 100:.2f}%, 
Maximum Gini Difference = {np.max(stability) * 100:.2f}%
Minimum Gini Difference = {np.min(stability) * 100:.2f}%
""")

 -- MODEL STABILITY --
Average Gini Difference = 0.85%, 
Maximum Gini Difference = 0.97%
Minimum Gini Difference = 0.62%



In [21]:
most_stable_model = np.argmin(stability)
most_stable_model

2

In [22]:
model_xgb_final = models[most_stable_model]

### Save Model

In [23]:
model_xgb_final.save_model('li_intent_model_2025-10-07.json')

### Load Model

In [24]:
model_xgb_final = xgb.Booster()
model_xgb_final.load_model('li_intent_model_2025-10-07.json')


### OOT Metrics

In [45]:
QUERY = f"""
    SELECT DISTINCT
        customer_id, snapshot_partition, sms_target, abli_target, final_target, {feature_query}
    FROM
        `abcd-dataplatform.abcd_data_science_app.li_intent_model_combined_features`
    WHERE
        snapshot_partition = '2025-05-31'
"""

oot_set = client.query(QUERY).to_dataframe()
oot_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5856022 entries, 0 to 5856021
Data columns (total 25 columns):
 #   Column                                 Dtype  
---  ------                                 -----  
 0   customer_id                            object 
 1   snapshot_partition                     dbdate 
 2   sms_target                             Int64  
 3   abli_target                            Int64  
 4   final_target                           Int64  
 5   IsParent                               object 
 6   age_bracket                            object 
 7   bank_with_highest_avg_balance          object 
 8   insurance_avg_topspend_amount          float64
 9   max_total_monthly_expense_L12M         float64
 10  latest_txn_date_L9M                    object 
 11  emi_repayment_avg_topspend_amount      float64
 12  most_frequent_bank                     object 
 13  max_monthly_surplus_L12M               float64
 14  max_monthly_bankbalance_L6M            float64
 15

In [46]:
oot_set = oot_set[oot_set[top_features].notna().any(axis=1)].reset_index(drop=True)
oot_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4966820 entries, 0 to 4966819
Data columns (total 25 columns):
 #   Column                                 Dtype  
---  ------                                 -----  
 0   customer_id                            object 
 1   snapshot_partition                     dbdate 
 2   sms_target                             Int64  
 3   abli_target                            Int64  
 4   final_target                           Int64  
 5   IsParent                               object 
 6   age_bracket                            object 
 7   bank_with_highest_avg_balance          object 
 8   insurance_avg_topspend_amount          float64
 9   max_total_monthly_expense_L12M         float64
 10  latest_txn_date_L9M                    object 
 11  emi_repayment_avg_topspend_amount      float64
 12  most_frequent_bank                     object 
 13  max_monthly_surplus_L12M               float64
 14  max_monthly_bankbalance_L6M            float64
 15

In [47]:
smaller_sets = [
    (oot_set.drop(columns=['sms_target', 'abli_target']).rename(columns={'final_target': 'target'}), 'Combined Target'),
    (oot_set.drop(columns=['sms_target', 'final_target']).rename(columns={'abli_target': 'target'}), 'LOB Target'),
    (oot_set.drop(columns=['final_target', 'abli_target']).rename(columns={'sms_target': 'target'}), 'SMS Target'),
]

In [48]:
reports = []
for df, snapshot in smaller_sets:
    print(df.shape)
    dmat = create_dmatrix(df[top_features + ['snapshot_partition']].copy(), date_cols=date_cols_filtered, categorical_cols=categorical_cols_filtered, label=df['target'].copy())
    # Predict probabilities
    y_pred = model_xgb_final.predict(dmat)

    # Extract true labels from DMatrix
    y_true = dmat.get_label()

    # Calculate AUC
    auc = roc_auc_score(y_true, y_pred)

    # Calculate Gini
    gini = 2 * auc - 1
    
    reports.append(Box(
        {
            'fold_snapshot': snapshot,
            'fold_size': df.snapshot_partition.count(),
            'fold_gini': gini,
            'fold_auc': auc
        }
    ))

(4966820, 23)
(4966820, 23)
(4966820, 23)


In [49]:
print('-- FOR OUT OF TIME VALIDATION --')
for report in reports:
    print_str = f"Fold Snapshot: {report.fold_snapshot} | Fold Size: {report.fold_size} | Fold Gini: {report.fold_gini * 100:.2f}%"
    print(print_str)

-- FOR OUT OF TIME VALIDATION --
Fold Snapshot: Combined Target | Fold Size: 4966820 | Fold Gini: 78.11%
Fold Snapshot: LOB Target | Fold Size: 4966820 | Fold Gini: 74.13%
Fold Snapshot: SMS Target | Fold Size: 4966820 | Fold Gini: 81.33%


In [50]:
dtrain = create_dmatrix(
    oot_set.drop(columns=['sms_target', 'abli_target', 'final_target']).copy(), 
    date_cols=date_cols_filtered, 
    categorical_cols=categorical_cols_filtered, 
    label=oot_set.rename(columns={'final_target': 'target'})['target'].copy()
)

In [51]:
y_test = dtrain.get_label()

y_pred_proba = model_xgb_final.predict(dtrain)
deciles = pd.qcut(pd.Series(y_pred_proba).rank(method='first'), q=10, labels=False, duplicates='raise') + 1
deciles_df = pd.DataFrame({
    'Probability': y_pred_proba,
    'Actual': y_test,
    'Decile': deciles,
    'Bin Size': 1
}).sort_values(['Decile', 'Probability'],ascending=False)

agg_deciles_df = deciles_df.groupby('Decile').agg({
    'Probability': ['mean', 'nunique'],
    'Actual':'sum',
    'Bin Size': 'sum'
}).reset_index().sort_values(['Decile'], ascending=False)

agg_deciles_df['response_rate'] = agg_deciles_df['Actual'] / agg_deciles_df['Bin Size']
agg_deciles_df['decile_recall'] = agg_deciles_df['Actual'] / y_test.sum()
agg_deciles_df['capture_rate'] = agg_deciles_df['Actual'].cumsum() * 100 / y_test.sum()
agg_deciles_df['Bin Size'] = agg_deciles_df.pop('Bin Size')
agg_deciles_df

Decile Probability          Actual response_rate decile_recall capture_rate  \
                mean nunique     sum                                            
9     10    0.003002  378929  4817.0      0.009698      0.694893    69.489326   
8      9    0.000727  314558   881.0      0.001774      0.127092    82.198502   
7      8    0.000440  338281   509.0      0.001025      0.073428    89.541260   
6      7    0.000297  260163   264.0      0.000532      0.038084    93.349686   
5      6    0.000219  168556   164.0      0.000330      0.023658    95.715523   
4      5    0.000178  153233   116.0      0.000234      0.016734    97.388924   
3      4    0.000155   75064    62.0      0.000125      0.008944    98.283325   
2      3    0.000134  115267    60.0      0.000121      0.008656    99.148872   
1      2    0.000111  115124    36.0      0.000072      0.005193    99.668205   
0      1    0.000085  129828    23.0      0.000046      0.003318   100.000000   

  Bin Size  
            
9   496682  
8   496682  
7   496682  
6   496682  
5   496682  
4   496682  
3   496682  
2   496682  
1   496682  
0   496682

In [52]:
dtrain = create_dmatrix(
    oot_set.drop(columns=['sms_target', 'abli_target', 'final_target']).copy(), 
    date_cols=date_cols_filtered, 
    categorical_cols=categorical_cols_filtered, 
    label=oot_set.rename(columns={'abli_target': 'target'})['target'].copy()
)

In [53]:
y_test = dtrain.get_label()

y_pred_proba = model_xgb_final.predict(dtrain)
deciles = pd.qcut(pd.Series(y_pred_proba).rank(method='first'), q=10, labels=False, duplicates='raise') + 1
deciles_df = pd.DataFrame({
    'Probability': y_pred_proba,
    'Actual': y_test,
    'Decile': deciles,
    'Bin Size': 1
}).sort_values(['Decile', 'Probability'],ascending=False)

agg_deciles_df = deciles_df.groupby('Decile').agg({
    'Probability': ['mean', 'nunique'],
    'Actual':'sum',
    'Bin Size': 'sum'
}).reset_index().sort_values(['Decile'], ascending=False)

agg_deciles_df['response_rate'] = agg_deciles_df['Actual'] / agg_deciles_df['Bin Size']
agg_deciles_df['decile_recall'] = agg_deciles_df['Actual'] / y_test.sum()
agg_deciles_df['capture_rate'] = agg_deciles_df['Actual'].cumsum() * 100 / y_test.sum()
agg_deciles_df['Bin Size'] = agg_deciles_df.pop('Bin Size')
agg_deciles_df

Decile Probability          Actual response_rate decile_recall capture_rate  \
                mean nunique     sum                                            
9     10    0.003002  378929  1805.0      0.003634      0.653512    65.351196   
8      9    0.000727  314558   356.0      0.000717      0.128892    78.240402   
7      8    0.000440  338281   204.0      0.000411      0.073860    85.626358   
6      7    0.000297  260163   130.0      0.000262      0.047067    90.333092   
5      6    0.000219  168556    99.0      0.000199      0.035844    93.917450   
4      5    0.000178  153233    55.0      0.000111      0.019913    95.908760   
3      4    0.000155   75064    36.0      0.000072      0.013034    97.212166   
2      3    0.000134  115267    40.0      0.000081      0.014482    98.660393   
1      2    0.000111  115124    25.0      0.000050      0.009051    99.565529   
0      1    0.000085  129828    12.0      0.000024      0.004345   100.000000   

  Bin Size  
            
9   496682  
8   496682  
7   496682  
6   496682  
5   496682  
4   496682  
3   496682  
2   496682  
1   496682  
0   496682

In [54]:
dtrain = create_dmatrix(
    oot_set.drop(columns=['sms_target', 'abli_target', 'final_target']).copy(), 
    date_cols=date_cols_filtered, 
    categorical_cols=categorical_cols_filtered, 
    label=oot_set.rename(columns={'sms_target': 'target'})['target'].copy()
)

In [55]:
y_test = dtrain.get_label()

y_pred_proba = model_xgb_final.predict(dtrain)
deciles = pd.qcut(pd.Series(y_pred_proba).rank(method='first'), q=10, labels=False, duplicates='raise') + 1
deciles_df = pd.DataFrame({
    'Probability': y_pred_proba,
    'Actual': y_test,
    'Decile': deciles,
    'Bin Size': 1
}).sort_values(['Decile', 'Probability'],ascending=False)

agg_deciles_df = deciles_df.groupby('Decile').agg({
    'Probability': ['mean', 'nunique'],
    'Actual':'sum',
    'Bin Size': 'sum'
}).reset_index().sort_values(['Decile'], ascending=False)

agg_deciles_df['response_rate'] = agg_deciles_df['Actual'] / agg_deciles_df['Bin Size']
agg_deciles_df['decile_recall'] = agg_deciles_df['Actual'] / y_test.sum()
agg_deciles_df['capture_rate'] = agg_deciles_df['Actual'].cumsum() * 100 / y_test.sum()
agg_deciles_df['Bin Size'] = agg_deciles_df.pop('Bin Size')
agg_deciles_df

Decile Probability          Actual response_rate decile_recall capture_rate  \
                mean nunique     sum                                            
9     10    0.003002  378929  3269.0      0.006582      0.731975    73.197495   
8      9    0.000727  314558   548.0      0.001103      0.122705    85.467979   
7      8    0.000440  338281   316.0      0.000636      0.070757    92.543663   
6      7    0.000297  260163   135.0      0.000272      0.030228    95.566505   
5      6    0.000219  168556    67.0      0.000135      0.015002    97.066727   
4      5    0.000178  153233    62.0      0.000125      0.013883    98.454994   
3      4    0.000155   75064    26.0      0.000052      0.005822    99.037170   
2      3    0.000134  115267    20.0      0.000040      0.004478    99.485001   
1      2    0.000111  115124    11.0      0.000022      0.002463    99.731300   
0      1    0.000085  129828    12.0      0.000024      0.002687   100.000000   

  Bin Size  
            
9   496682  
8   496682  
7   496682  
6   496682  
5   496682  
4   496682  
3   496682  
2   496682  
1   496682  
0   496682

### Model Evaluation and Understanding

In [36]:
X_train = train_set.loc[:, top_features + ['snapshot_partition']]
y_train = train_set.loc[:, 'target']
dtrain = create_dmatrix(X_train, date_cols=date_cols_filtered, categorical_cols=categorical_cols_filtered, label=y_train)

In [37]:
train_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14790426 entries, 0 to 14790425
Data columns (total 20 columns):
 #   Column                                 Dtype   
---  ------                                 -----   
 0   IsParent                               category
 1   age_bracket                            category
 2   bank_with_highest_avg_balance          category
 3   insurance_avg_topspend_amount          float64 
 4   max_total_monthly_expense_L12M         float64 
 5   latest_txn_date_L9M                    float64 
 6   emi_repayment_avg_topspend_amount      float64 
 7   most_frequent_bank                     category
 8   max_monthly_surplus_L12M               float64 
 9   max_monthly_bankbalance_L6M            float64 
 10  bank_w_highest_cc_limit                category
 11  min_investment_amt_L3M                 float64 
 12  others_insurance_count_of_policies     Int64   
 13  upi_avg_topspend_amount                float64 
 14  provident_fund_avg_investment_am

In [38]:
%%time
explainer = shap.TreeExplainer(model_xgb_final, feature_perturbation="tree_path_dependent")
shap_values = explainer.shap_values(dtrain, approximate=True)

CPU times: user 25min 50s, sys: 1.05 s, total: 25min 51s
Wall time: 51.5 s


In [39]:
def shap_numeric_summary(shap_values, X, sample_size=50000, save=False, out_dir="./shap_plots"):
    """
    SHAP beeswarm summary plot for numerical features only.
    
    Parameters
    ----------
    shap_values : np.ndarray
        SHAP values matrix (n_samples, n_features).
    X : pd.DataFrame
        Training features DataFrame.
    sample_size : int, optional
        Number of samples to randomly choose for plotting (default=50,000).
    save : bool, optional
        If True, saves the plot instead of showing it.
    out_path : str, optional
        Path to save the plot (if save=True).
    """
    # Identify numerical columns
    num_cols = X.select_dtypes(include=["float64", "Int64"]).columns
    num_indices = [list(X.columns).index(c) for c in num_cols]

    # Subsample
    if sample_size and len(X) > sample_size:
        sample_idx = np.random.choice(X.index, size=sample_size, replace=False)
    else:
        sample_idx = X.index
    
    # Slice SHAP values + feature values
    shap_values_num = shap_values[sample_idx][:, num_indices]
    features_num = X.loc[sample_idx, num_cols].astype(float)

    # Plot
    shap.summary_plot(
        shap_values_num,
        features_num,
        feature_names=num_cols,
        show=not save  # suppress auto-show if saving
    )

    if save:
        import os
        os.makedirs(out_dir, exist_ok=True)
        plt.savefig(f"{out_dir}/shap_numeric.png", bbox_inches="tight")
        plt.close()


In [40]:
def shap_category_beeswarm(feature, shap_values, X, top_n=15, sample_size=50000, save=False, out_dir="./shap_plots/"):
    """
    Beeswarm-style SHAP visualization for categorical features.
    """
    # Subsample if dataset is huge
    if sample_size and len(X) > sample_size:
        sample_idx = np.random.choice(X.index, size=sample_size, replace=False)
        X = X.loc[sample_idx]
        shap_values = shap_values[sample_idx]
    
    col_idx = list(X.columns).index(feature)
    shap_vals = shap_values[:, col_idx]
    
    df = pd.DataFrame({
        "category": X[feature].astype(str),
        "shap": shap_vals
    })
    
    # Rank by mean |SHAP| and keep top-N categories
    top_cats = (
        df.groupby("category")["shap"]
        .apply(lambda x: np.mean(np.abs(x)))
        .nlargest(top_n)
        .index
    )
    df = df[df["category"].isin(top_cats)]
    
    plt.figure(figsize=(10, 6))
    sns.stripplot(
        x="shap", y="category", data=df,
        jitter=0.3, alpha=0.5, orient="h"
    )
    plt.title(f"Beeswarm-style SHAP for {feature} (Top {top_n} categories)")
    plt.xlabel("SHAP value (impact on model output)")
    
    if save:
        import os
        os.makedirs(out_dir, exist_ok=True)
        plt.savefig(f"{out_dir}/shap_{feature}.png", bbox_inches="tight")
        plt.close()
    else:
        plt.show()

In [41]:
def shap_feature_importance_bar(shap_values, X, top_n=None, save=False, out_dir="./shap_plots", out_path="shap_importance_bar.png"):
    """
    SHAP bar summary plot (mean absolute SHAP value per feature).
    
    Parameters
    ----------
    shap_values : np.ndarray
        SHAP values matrix (n_samples, n_features).
    X : pd.DataFrame
        Training features DataFrame.
    top_n : int, optional
        Show only top-N features by importance (default=None, show all).
    save : bool, optional
        If True, saves the plot instead of displaying it.
    out_path : str, optional
        Path to save the plot (if save=True).
    """
    shap.summary_plot(
        shap_values,
        X,
        plot_type="bar",
        max_display=top_n if top_n else None,
        show=not save  # suppress auto-show if saving
    )

    if save:
        import os
        os.makedirs(out_dir, exist_ok=True)
        plt.savefig(f"{out_dir}/{out_path}", bbox_inches="tight")
        plt.close()


In [42]:
shap_numeric_summary(shap_values, train_features, sample_size=50000, save=True)

In [43]:
cat_cols = train_features.select_dtypes(include=["category"]).columns

for feature in cat_cols:
    shap_category_beeswarm(feature, shap_values, train_features, top_n=15, sample_size=50000, save=True)

In [44]:
shap_feature_importance_bar(shap_values, train_features, save=True)

In [36]:
shap_abs = pd.DataFrame(shap_values, columns=train_features.columns).abs().mean()
shap_abs.loc['dha_interacted']

0.0026672482

In [73]:
train_features.columns

Index(['age_bracket', 'bank_with_highest_avg_balance', 'most_frequent_bank',
       'latest_txn_date_ALL', 'insurance_avg_topspend_amount',
       'avg_monthly_spend_amount_L1M', 'min_monthly_surplus_L9M',
       'max_monthly_bankbalance_L3M', 'max_monthly_surplus_L12M',
       'emi_repayment_count_topspend', 'avg_investment_amt_L3M',
       'max_spend_category', 'min_monthly_bankbalance_ALL',
       'insurance_count_topspend', 'p2a_avg_topspend_amount',
       'median_monthly_utility_paid_amount_L6M', 'min_investment_amt_L3M',
       'insurance_avg_due_amount', 'p2p_avg_topspend_amount',
       'avg_monthly_investment_count_L3M'],
      dtype='object')